# Multi-Agent RAG System Using Crew AI for Tutorial Question Answering

### Use Case

In today's learning process, especially when exploring new frameworks or tools, we often rely on multiple resources. For example, when watching a YouTube tutorial, the video often contains valuable insights. Additionally, the tutorial description usually includes a link to a GitHub repository, and there is often an accompanying documentation website. If we have further questions, we can search the internet for more specific answers. 

However, the challenge arises when we need to seek answers to specific questions. Typically, we have to manually search through the YouTube video, navigate the GitHub repository, explore the documentation, and search the internet individually. This can be time-consuming and inefficient.

This **Multi Agent - Retrieval-Augmented Generation (RAG) system** aims to streamline that process. Using Crew AI, we can automatically extract relevant information from:

- The YouTube video
- The GitHub repository
- The documentation website
- An internet search

The system compiles the extracted information to generate a comprehensive answer to the user's query, thereby enhancing the learning process and providing more efficient access to knowledge. 

With this multi-agent RAG system, you no longer need to search each resource manually. Instead, the system will retrieve and compile the most relevant information from all available sources to answer your query, providing a smoother and faster learning experience.

In [1]:
from crewai import Agent
from crewai import Task 
from crewai import Crew
from crewai import Process
from crewai import LLM
from crewai_tools import tool
from crewai_tools import GithubSearchTool
from crewai_tools import YoutubeVideoSearchTool
from crewai_tools import WebsiteSearchTool
from duckduckgo_search import DDGS
import os
from IPython.display import Markdown
from dotenv import load_dotenv
load_dotenv()

sagemaker.config INFO - Not applying SDK defaults from location: /opt/homebrew/share/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/devanshsrivastav/Library/Application Support/sagemaker/config.yaml


True

In [2]:
# llm=LLM(model="ollama/llama3.1", base_url="http://localhost:11434")

In [3]:
gh_token = os.getenv("GITHUB_TOKEN")

In [4]:
youtube_search_tool = YoutubeVideoSearchTool(
    youtube_video_url='https://www.youtube.com/watch?v=7GhWXODugWM&t=3516s',
    config = dict(
        llm = dict(
            provider = "ollama",
            config = dict(
                model = "mistral"
            )
        ),
        embedder = dict(
            provider = "ollama",
            config = dict(
                model = "mistral"
            )
        )
    ),
)

Inserting batches in chromadb: 100%|██████████| 1/1 [00:42<00:00, 42.11s/it]


In [5]:
github_search_tool = GithubSearchTool(
    github_repo='bhancockio/crewai-rag-deep-dive',
    content_types=['code', 'issue'],
    gh_token = gh_token
)

Searching data from github:   0%|          | 0/2 [00:00<?, ?it/s]
Loading issue from github: 0it [00:00, ?it/s]
Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]


In [6]:
website_search_tool = WebsiteSearchTool(
    website='https://docs.crewai.com/how-to/Create-Custom-Tools/',
)

Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]


In [7]:
@tool("Internet Search Tool")
def internet_search_tool(query: str) -> list:
    """Search Internet for relevant information based on a query."""
    ddgs = DDGS()
    results = ddgs.text(keywords=query, region='wt-wt', safesearch='moderate', max_results=15)
    return results

In [8]:
youtube_search_agent = Agent(
    role="YouTube Video Search Agent",
    goal="Search for specific information within a YouTube video and return the most relevant data.",
    allow_delegation=False,
    verbose=True,
    # llm=llm,
    backstory=(
        """
        The YouTube Video Search Agent specializes in searching through YouTube video content to
        find information based on user-provided queries. The agent can navigate through timestamps 
        and process video transcripts or audio to deliver concise results.
        """
    ),
    tools=[youtube_search_tool]
)

In [9]:
github_search_agent = Agent(
    role="GitHub Repository Search Agent",
    goal="Search for specific information within a GitHub repository and return relevant data from code or issues.",
    allow_delegation=False,
    verbose=True,
    # llm=llm,
    backstory=(
        """
        The GitHub Repository Search Agent specializes in searching through GitHub repositories 
        for code, issues, and other content based on user queries. The agent can analyze code 
        structure, issues, and comments to provide relevant information from the repository.
        """
    ),
    tools=[github_search_tool]
)

In [10]:
website_search_agent = Agent(
    role="Website Documentation Search Agent",
    goal="Search for specific information within the Crew AI documentation and provide concise and relevant information.",
    allow_delegation=False,
    verbose=True,
    # llm=llm, 
    backstory=(
        """
        The Website Documentation Search Agent specializes in searching through the Crew AI documentation 
        website to find information related to user queries. The agent can navigate the documentation pages,
        search for keywords, and return relevant sections in a clear, concise manner.
        """
    ),
    tools=[website_search_tool]
)


In [11]:
internet_search_agent = Agent(
    role="Internet Search Agent",
    goal="Perform a search on the internet and return relevant information based on the user's query.",
    allow_delegation=False,
    verbose=True,
    # llm=llm,
    backstory=(
        """
        The Internet Search Agent is skilled in conducting web searches to find the most relevant and 
        up-to-date information. The agent uses an internet search tool to retrieve and summarize 
        information based on a provided query.
        """
    ),
    tools=[internet_search_tool]
)


In [12]:
youtube_search_task = Task(
    description=(
        """
        Search the YouTube video for information related to the query: '{user_question}'.
        Navigate through the video, and based on timestamps, transcripts, or audio data, return 
        the most relevant information to answer the query.
        """
    ),
    expected_output="""
        A clear and concise summary of the relevant portion of the video that matches the search query.
        The summary should include the timestamp and key points related to the query.
    """,
    tools=[youtube_search_tool],
    agent=youtube_search_agent,
)


In [13]:
github_search_task = Task(
    description=(
        """
        Search the GitHub repository for content related to the query: '{user_question}'.
        This includes searching through both code and issues in the repository. Provide a detailed 
        summary of the most relevant code snippets or issue discussions related to the query.
        """
    ),
    expected_output="""
        A detailed summary of the relevant code or issue from the repository, based on the search query.
        Include links to specific files, lines of code, or issue threads, along with a concise explanation
        of how the content relates to the query.
    """,
    tools=[github_search_tool],
    agent=github_search_agent,
)

In [14]:
website_search_task = Task(
    description=(
        """
        Search the Crew AI documentation website for information related to the query: '{user_question}'.
        Provide a clear summary of the relevant documentation sections, with links to the exact pages or sections 
        where the information can be found.
        """
    ),
    expected_output="""
        A concise summary of the relevant section(s) from the Crew AI documentation based on the search query,
        including direct links to specific documentation pages or subsections.
    """,
    tools=[website_search_tool],
    agent=website_search_agent,
)


In [15]:
internet_search_task = Task(
    description=(
        """
        Search the internet for relevant information using the query: '{user_question}'.
        Provide a list of relevant results, summarizing the key findings and including direct links 
        to the most useful sources.
        """
    ),
    expected_output="""
        A list of relevant results from the internet based on the search query. Each result should 
        include a brief summary and a direct link to the source.
    """,
    tools=[internet_search_tool],
    agent=internet_search_agent,
)


In [16]:
manager = Agent(
    role="Master Agent",
    goal="Oversee the research process and manage the specialized agents to deliver comprehensive, high-quality outputs.",
    backstory="Experienced project manager coordinating the efforts of agents across different domains like web, GitHub, YouTube, and internet search.",
    allow_delegation=True,
    cache=True,
    verbose=True,
    # llm=llm,
)


In [17]:
crew = Crew(
    agents=[youtube_search_agent, github_search_agent, website_search_agent, internet_search_agent],
    tasks=[youtube_search_task, github_search_task, website_search_task, internet_search_task],
    manager_agent=manager,
    process=Process.hierarchical
)

In [18]:
user_question = "How to create a custom tool in crewAI?"

In [19]:
result = crew.kickoff(
    inputs={"user_question": user_question}
)

# Agent: Master Agent
## Task: 
        Search the YouTube video for information related to the query: 'How to create a custom tool in crewAI?'.
        Navigate through the video, and based on timestamps, transcripts, or audio data, return 
        the most relevant information to answer the query.
        
# Agent: YouTube Video Search Agent
## Task: Search for YouTube videos related to 'How to create a custom tool in crewAI'. Navigate through the video, and based on timestamps, transcripts, or audio data, return the most relevant information to answer the query.


# Agent: YouTube Video Search Agent
## Thought: I need to search for YouTube videos related to "How to create a custom tool in crewAI" and gather the most relevant information, including timestamps and key points.
## Using tool: Search a Youtube Video content
## Tool Input: 
"{\"search_query\": \"How to create a custom tool in crewAI\"}"
## Tool Output: 
Relevant Content:
saying like hey your task is to fill out this conte

In [20]:
print(result)

Here’s a detailed guide on how to create a custom tool in Crew AI, taking insights from various relevant sources.

1. **Creating a Custom Tool**:
   To create a custom tool in Crew AI, you can follow these steps:

   - **Inherit from BaseTool**: Start by inheriting from the `BaseTool` class. This is the foundation upon which your custom tool will be built.

     ```python
     from crewai_tools import BaseTool

     class MyCustomTool(BaseTool):
         name: str = "Name of my tool"
         description: str = "What this tool does. It's vital for effective utilization."

         def _run(self, argument: str) -> str:
             # Your tool's logic here
             ...
     ```

   - **Define Tool Specifications**: Outline the functionality your tool will deliver, including input/output specifications (documentation reference: [Creating Custom Tools in crewAI](https://docs.crewai.com/how-to/Create-Custom-Tools/)).

2. **Use a Tool Decorator**:
   Alternatively, you can use the tool 

In [32]:
Markdown(result.raw)

Here’s a detailed guide on how to create a custom tool in Crew AI, taking insights from various relevant sources.

1. **Creating a Custom Tool**:
   To create a custom tool in Crew AI, you can follow these steps:

   - **Inherit from BaseTool**: Start by inheriting from the `BaseTool` class. This is the foundation upon which your custom tool will be built.

     ```python
     from crewai_tools import BaseTool

     class MyCustomTool(BaseTool):
         name: str = "Name of my tool"
         description: str = "What this tool does. It's vital for effective utilization."

         def _run(self, argument: str) -> str:
             # Your tool's logic here
             ...
     ```

   - **Define Tool Specifications**: Outline the functionality your tool will deliver, including input/output specifications (documentation reference: [Creating Custom Tools in crewAI](https://docs.crewai.com/how-to/Create-Custom-Tools/)).

2. **Use a Tool Decorator**:
   Alternatively, you can use the tool decorator for a simpler implementation. This allows you to create a Tool object directly.

   ```python
   from crewai_tools import tool

   @tool("Name of my tool")
   def my_tool(question: str) -> str:
       """Clear description for tool usage."""
       ...
   ```

   This method is beneficial for straightforward implementations without subclassing.

3. **Additional Documentation and Resources**:
   Here are some resources that can provide additional context and examples:

   - Official Documentation: [Creating Custom Tools in Crew AI](https://docs.crewai.com/how-to/Create-Custom-Tools/)
   - Help Center: [Creating Custom Tools for CrewAI Agents](https://help.crewai.com/how-can-i-create-custom-tools-for-my-crewai-agents)
   - GitHub Documentation: [crewAI Custom Tools Documentation](https://github.com/crewAIInc/crewAI/blob/main/docs/how-to/Create-Custom-Tools.md)
   - Comprehensive Guide to CrewAI Tools: [CrewAI Tools Documentation](https://docs.crewai.com/core-concepts/Tools/)

4. **Testing & Deployment**:
   - Rigorously test your custom tool in multiple scenarios to ensure it meets performance specifications.
   - Upon successful testing, deploy the tool within your Crew AI environment.

5. **Best Practices**:
   - Always document your tool’s functionality and usage guidelines for ease of reference in the future.
   - Consider leveraging existing tools within CrewAI to save time and improve functionality.

6. **Community Insights and Examples**:
   - For deeper insights, check Medium articles and YouTube tutorials that present practical examples and step-by-step guides on utilizing CrewAI, such as platforms like [Deep Dive into CrewAI](https://medium.com/@tarekeesa7/deep-dive-into-crewai-with-examples-cd63b47e0bd3).

By following these steps and exploring the linked resources, you will gain a thorough understanding of creating a custom tool in Crew AI. If you have further questions, feel free to reach out to the community forums or the CrewAI support team.